# Find the change in shot distance after a certain amount of misses.

In [3]:
from nba_api.stats.endpoints import shotchartdetail
import json
import requests
import pandas as pd

# Grab player and team ids
# Load teams file
teams = json.loads(requests.get('https://raw.githubusercontent.com/bttmly/nba/master/data/teams.json').text)
# Load players file
players = json.loads(requests.get('https://lifeislinear.davidson.edu/sports/player_dicts.json').text)

### ID Helper functions.

In [4]:
# Get team ID based on team name
def get_team_id(teamInput):
    for team in teams:
        if team['teamName'] == teamInput:
            return team['teamId']
    return -1
# Get player ID based on player name
def get_player_id(first, last):
    for player in players:
        if player['firstName'] == first and player['lastName'] == last:
            return player['playerId']
    return -1
# Get Name based on player id
def get_player_name(playerId):
    for player in players:
        if player['playerId']==playerId:
            return player['firstName'], player['lastName']
    return -1

In [5]:
# Load the data
lebron_json = shotchartdetail.ShotChartDetail(
            # team_id = get_team_id('Los Angeles Lakers'),
            team_id = 0,
            player_id = get_player_id('LeBron', 'James'),
            context_measure_simple = 'FGA',
            season_nullable = '2016-17',
            season_type_all_star = ['Regular Season', 'Playoffs'])

data = json.loads(lebron_json.get_json())

df = pd.concat(
    pd.DataFrame(shot, columns=data['resultSets'][0]['headers'])
    for shot in [data['resultSets'][0]['rowSet']]
)

df['PREV_GAME'] = df["GAME_ID"].shift(1).fillna(0)

miss_streak = 0
for index, row in df.iterrows():
    if row['SHOT_MADE_FLAG'] == 0:
        if row['GAME_ID'] != row['PREV_GAME'] and row['PREV_GAME'] != 0:
            miss_streak = 0
        else:
            miss_streak += 1
    else:
        miss_streak = 0
    
    df.at[index, 'MISS_STREAK'] = miss_streak
    
df.head(100)

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM,PREV_GAME,MISS_STREAK
0,Shot Chart Detail,0021600001,8,2544,LeBron James,1610612739,Cleveland Cavaliers,1,10,59,...,11,-79,80,1,1,20161025,CLE,NYK,0,0.0
1,Shot Chart Detail,0021600001,29,2544,LeBron James,1610612739,Cleveland Cavaliers,1,9,27,...,24,-166,185,1,0,20161025,CLE,NYK,0021600001,1.0
2,Shot Chart Detail,0021600001,40,2544,LeBron James,1610612739,Cleveland Cavaliers,1,8,16,...,25,133,213,1,0,20161025,CLE,NYK,0021600001,2.0
3,Shot Chart Detail,0021600001,69,2544,LeBron James,1610612739,Cleveland Cavaliers,1,5,56,...,0,9,-1,1,1,20161025,CLE,NYK,0021600001,0.0
4,Shot Chart Detail,0021600001,98,2544,LeBron James,1610612739,Cleveland Cavaliers,1,3,37,...,0,0,1,1,1,20161025,CLE,NYK,0021600001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Shot Chart Detail,0021600080,363,2544,LeBron James,1610612739,Cleveland Cavaliers,3,4,53,...,25,-24,254,1,0,20161105,PHI,CLE,0021600080,2.0
96,Shot Chart Detail,0021600080,395,2544,LeBron James,1610612739,Cleveland Cavaliers,3,2,18,...,2,-20,-6,1,1,20161105,PHI,CLE,0021600080,0.0
97,Shot Chart Detail,0021600080,475,2544,LeBron James,1610612739,Cleveland Cavaliers,4,5,26,...,1,-11,11,1,0,20161105,PHI,CLE,0021600080,1.0
98,Shot Chart Detail,0021600080,480,2544,LeBron James,1610612739,Cleveland Cavaliers,4,4,56,...,0,0,1,1,1,20161105,PHI,CLE,0021600080,0.0


In [6]:
from scipy import stats

df['NEXT_SHOT_DISTANCE'] = df['SHOT_DISTANCE'].shift(-1).fillna(0)

one_miss = df[(df['MISS_STREAK'] == 1) &
              (df['GAME_ID'] == df['PREV_GAME'])]

two_miss = df[(df['MISS_STREAK'] == 2) &
              (df['GAME_ID'] == df['PREV_GAME'])]

three_miss = df[(df['MISS_STREAK'] >= 3) &
              (df['GAME_ID'] == df['PREV_GAME'])]

one_change = one_miss['SHOT_DISTANCE'].mean() - one_miss['NEXT_SHOT_DISTANCE'].mean()
two_change = two_miss['SHOT_DISTANCE'].mean() - two_miss['NEXT_SHOT_DISTANCE'].mean()
three_change = three_miss['SHOT_DISTANCE'].mean() - three_miss['NEXT_SHOT_DISTANCE'].mean()


print(one_change)
print(two_change)
print(three_change)


# Statistical significance
t, p = stats.ttest_rel(one_miss['SHOT_DISTANCE'], one_miss['NEXT_SHOT_DISTANCE'])
print(f'T Statistic {t}, p-value {p}')
t, p = stats.ttest_rel(two_miss['SHOT_DISTANCE'], two_miss['NEXT_SHOT_DISTANCE'])
print(f'T Statistic {t}, p-value {p}')
t, p = stats.ttest_rel(three_miss['SHOT_DISTANCE'], three_miss['NEXT_SHOT_DISTANCE'])
print(f'T Statistic {t}, p-value {p}')

5.700296735905043
3.8014184397163113
6.621951219512194
T Statistic 7.523204946120523, p-value 4.93520197042505e-13
T Statistic 3.148259617450707, p-value 0.002007715470072515
T Statistic 4.436840447337986, p-value 2.849990886588082e-05
